# Project Euler

## Problem 51

### Prime digit replacements

In [65]:
import itertools

limit_pow = 8
limit = 10**limit_pow

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]

primes = prime_sieve(limit)

def create_number(n, d):
    return int(n.replace('*', str(d)))
    
def create_base(n, digits):
    s = str(n)
    base = ''
    digs = set([s[d-1] for d in digits])
    if len(digs) != 1:
        return base
    l = 0
    for d in digits:
        base += s[l:d-1] + '*'
        l = d
    base += s[l:]
    return base
    
def prime_family(n):
    prime_family = []
    start = 0
    if n[0] == '*':
        start = 1
    for d in range(start,10):
        candidate = create_number(n, d)
        if candidate in primes:
            prime_family.append(candidate)
    
    return prime_family

size = 8
solved = False

combinations = {}
for i in range(1, limit_pow+1):
    if i not in combinations:
        combinations[i] = []
    for k in range(1,i):    
        combinations[i].extend([list(j) for j in list(itertools.combinations(range(1,i), k))])

while not solved:
    bases = {}
    for n in primes:
        for d in combinations[len(str(n))]:
            base = create_base(n, d)
            if base != '':
                if base in bases:
                    bases[base].add(n)
                    if len(bases[base]) >= size:
                        solved = True
                        break
                else:
                    bases[base] = set([n])
        if solved:
            break

bases[base]

{121313, 222323, 323333, 424343, 525353, 626363, 828383, 929393}

## Problem 52

### Permuted multiples

In [79]:
def digits(n):
    return sorted(list(str(n)))

def multiple_digits(n):
    l = []
    for i in range(1,7):
        l.append(digits(i*n))
    return l
        
def check(l):
    valid = True
    for i in range(0,len(l)-1):
        if l[i] != l[i+1]:
            valid = False
            break
    return valid

n = 1
while True:
    if check(multiple_digits(n)):
        break
    n += 1
    
n

142857

## Problem 53

### Combinatoric selections

In [89]:
import math

def combinations(n,r):
    return int(math.factorial(n)/(math.factorial(r)*math.factorial(n-r)))

def count_combinations_for_n(n,limit):
    a = 0
    for r in range(1,n+1):
        if combinations(n,r) >= limit:
            a += 1
    return a

a = 0
for n in range(1,101):
    a += count_combinations_for_n(n, 1000000)
    
a

4075

## Problem 54

### Poker hands

In [186]:
with open("p054_poker.txt") as f:
    s = f.read()
    
games = s.split('\n')
hands = {}

card_values = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']

for i in range(0, len(games)):
    cards = games[i].split(' ')
    hands[i] = {}
    hands[i][1] = cards[0:5]
    hands[i][2] = cards[5:10]
    
del hands[1000]

def is_royal_flush(hand):
    required_values = set(card_values[8:13])
    suites = set()
    values = set()
    for card in hand:
        values.add(card[0])
        suites.add(card[1])
    return len(suites) == 1 and required_values == values

def is_straight_flush(hand):
    suites = set()
    values = set()
    min_index = 13
    for card in hand:
        values.add(card[0])
        suites.add(card[1])
        i = card_values.index(card[0])
        if i < min_index:
            min_index = i
    if len(suites) != 1:
        return False
    return values == set(card_values[min_index:min_index+5])

def is_four_of_a_kind(hand):
    values = []
    for card in hand:
        values.append(card[0])
    for v in set(values):
        if values.count(v) == 4:
            return (True, v)
    return (False, 0)

def is_full_house(hand):
    values = []
    for card in hand:
        values.append(card[0])
    if len(set(values)) == 2: # either this, or four of a kind
        for v in set(values):
            if values.count(v) == 3:
                v1 = v
            if values.count(v) == 2:
                v2 = v
        return (True, v1, v2)
    return (False, 0, 0)

def is_flush(hand):
    suites = set()
    for card in hand:
        suites.add(card[1])
    return len(suites) == 1

def is_straight(hand):
    values = set()
    min_index = 13
    for card in hand:
        values.add(card[0])
        i = card_values.index(card[0])
        if i < min_index:
            min_index = i
    return values == set(card_values[min_index:min_index+5])

def is_three_of_a_kind(hand):
    values = []
    for card in hand:
        values.append(card[0])
    for v in set(values):
        if values.count(v) == 3:
            return (True, v)
    return (False, 0)

def is_two_pairs(hand):
    values = []
    pairs = []
    for card in hand:
        values.append(card[0])
    for v in set(values):
        if values.count(v) == 2:
            pairs.append(v)
    if len(pairs) == 2:
        return (True, sorted(pairs, reverse=True))
    return (False, [])

def is_pair(hand):
    values = []
    for card in hand:
        values.append(card[0])
    for v in set(values):
        if values.count(v) == 2:
            return (True, v)
    return (False, 0)

def card_ranking(hand):
    return sorted([card_values.index(card[0]) for card in hand], reverse=True)

def highest_rank(hand):
    if is_royal_flush(hand):
        return (21,0)
    if is_straight_flush(hand):
        return (20, 0)
    is_foak = is_four_of_a_kind(hand)
    if is_foak[0]:
        return (19, is_foak[1])
    is_fh = is_full_house(hand)
    if is_fh[0]:
        return (18,is_fh[1],is_fh[2])
    if is_flush(hand):
        return (17, 0)
    if is_straight(hand):
        return (16, 0)
    is_toak = is_three_of_a_kind(hand)
    if is_toak[0]:
        return (15, is_toak[1])
    is_tp = is_two_pairs(hand)
    if is_tp[0]:
        return (14, is_tp[1][0], is_tp[1][1])
    is_ip = is_pair(hand)
    if is_ip[0]:
        return (13, is_ip[1])
    else:
        return (max(card_ranking(hand)), 0)
    
def winner_by_highest(hand1, hand2):
    rank1 = card_ranking(hand1)
    rank2 = card_ranking(hand2)
    while len(rank1) != 0 and rank1[0] == rank2[0]:
        rank1 = rank1[1:len(rank1)]
        rank2 = rank2[1:len(rank2)]
    if rank1[0] > rank2[0]:
        return 1
    else:
        return 2
    
def determine_winner(hand1, hand2):
    rank1 = highest_rank(hand1)
    rank2 = highest_rank(hand2)
    if rank1[0] > rank2[0]:
        return 1
    elif rank2[0] > rank1[0]:
        return 2
    elif rank1[0] in [19,18,15,14,13]:
        if card_values.index(rank1[1]) > card_values.index(rank2[1]):
            return 1
        elif card_values.index(rank2[1]) > card_values.index(rank1[1]):
            return 2
        elif rank1[0] in [18,14]:
            if card_values.index(rank1[2]) > card_values.index(rank2[2]):
                return 1
            elif card_values.index(rank2[2]) > card_values.index(rank1[2]):
                return 2
    return winner_by_highest(hand1, hand2)
    
wins = 0
for h in hands:
    winner = determine_winner(hands[h][1], hands[h][2])
    if winner == 1:
        wins += 1
        
wins

376

## Problem 55

### Lychrel numbers

In [191]:
def reverse_add(n):
    return n + int(str(n)[::-1])

def is_palindrome(n):
    return str(n) == str(n)[::-1]

def is_lychrel(n):
    for i in range(0,50):
        n = reverse_add(n)
        if is_palindrome(n):
            return False
    return True


a = 0
for n in range(1,10001):
    if is_lychrel(n):
        a += 1
        
a

249

## Problem 56

### Powerful digit sum

In [195]:
def digit_sum(n):
    return sum([int(i) for i in list(str(n))])

max_ds = 0
for a in range(0,100):
    for b in range(0,100):
        ds = digit_sum(a**b)
        if ds > max_ds:
            max_ds = ds
            
max_ds

972

## Problem 57

### Square root convergents

In [24]:
def next_base(base):
    return (base[1], base[0]+2*base[1])

def calculate_root(base):
    return (base[0]+base[1],base[1])

def evaluate(root):
    return len(str(root[0])) > len(str(root[1]))

base = (1,2)
count = 0
for i in range(1000):
    root = calculate_root(base)
    if evaluate(root):
        count += 1
    base = next_base(base)
    
count

153

## Problem 58

### Spiral primes

In [85]:
limit_pow = 9
limit = 10**limit_pow

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]

primes = frozenset(prime_sieve(limit))

def diagonal_size(side_length):
    return 2*side_length-1

def count_corner_primes(side_length):
    p = 0
    for n in range(1, 4):
        d = side_length**2 - n*(side_length-1)
        if d in primes:
            p += 1
    return p

side_length = 3
diagonal_primes = count_corner_primes(side_length)
while not diagonal_primes/diagonal_size(side_length) < 0.1:
    side_length += 2
    diagonal_primes += count_corner_primes(side_length)

side_length  

26241

## Problem 59

### XOR decryption

In [142]:
with open("p059_cipher.txt") as f:
    cipher = f.read()
    
text = [int(c) for c in cipher.split(',')]

def decrypt(text, key):
    decrypted = []
    for i in range(0, len(text)):
        decrypted.append(text[i] ^ key[i%3])
    return decrypted

keys = list(itertools.product(range(ord('a'), ord('z')+1), repeat=3))
for key in keys:
    decrypted = decrypt(text, key)
    restored = ''.join([chr(c) for c in decrypted])
    if ' the ' in restored:
        break
    
sum(decrypted)

107359

## Problem 60

### Prime pair sets

In [244]:
import itertools

limit_pow = 9
limit = 10**limit_pow
target_length = 5

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]

primes = frozenset(prime_sieve(limit))
base_primes = [x for x in primes if x < 10**(limit_pow/2 + 1)]
base_primes.remove(2)
base_primes.remove(5)

found = []
index = 0
while len(found) < target_length:
    if len(base_primes) == index:
        index = base_primes.index(found[-1]) + 1
        found = found[:-1]
    match = True
    n = base_primes[index]
    for f in found:
        if (int(str(f)+str(n)) not in primes) or (int(str(n)+str(f)) not in primes):
            match = False
            break
    if match:
        found.append(n)
    index += 1

sum(found)

26033

## Problem 61

### Cyclical figurate numbers

In [515]:
import itertools

triangles = []
squares = []
pentagonals = []
hexagonals = []
heptagonals = []
octagonals = []

n = 1
while True:
    triangle   = int(n*(n+1)/2)
    square     = int(n*n)
    pentagonal = int(n*(3*n-1)/2)
    hexagonal  = int(n*(2*n-1))
    heptagonal = int(n*(5*n-3)/2)
    octagonal  = int(n*(3*n-2))
    
    if triangle >= 10000:
        break

    if triangle >= 1000 and triangle < 10000:
        triangles.append((str(triangle)[0:2], str(triangle)[2:]))
    if square >= 1000 and square < 10000:
        squares.append((str(square)[0:2], str(square)[2:]))
    if pentagonal >= 1000 and pentagonal < 10000:
        pentagonals.append((str(pentagonal)[0:2], str(pentagonal)[2:]))
    if hexagonal >= 1000 and hexagonal < 10000:
        hexagonals.append((str(hexagonal)[0:2], str(hexagonal)[2:]))
    if heptagonal >= 1000 and heptagonal < 10000:
        heptagonals.append((str(heptagonal)[0:2], str(heptagonal)[2:]))
    if octagonal >= 1000 and octagonal < 10000:
        octagonals.append((str(octagonal)[0:2], str(octagonal)[2:]))
        
    n += 1
    
perms = list(itertools.permutations([triangles, squares, pentagonals, hexagonals, heptagonals, octagonals], 6))
perms = perms[:int(len(perms)/6)]
    
can_work = []
for p in perms:
    options = dict()
    for i in range(0,6):
        options[i] = dict()
        for n in p[i]:
            if n[0] not in options[i]:
                options[i][n[0]] = []
            options[i][n[0]].extend([m for m in p[(i+1)%6] if n[1] == m[0]])
            if len(options[i][n[0]]) == 0:
                del options[i][n[0]]
        
    for k in range(0,6):
        if len(options) == 6:
            for i in range(0,6):
                for n in options[i]:
                    options[i][n] = [m for m in options[i][n] if (m[0] in options[(i+1)%6]) and (m[1] in options[(i+2)%6])]
                options[i] = {n:options[i][n] for n in options[i] if len(options[i][n]) != 0}
            options = {k:options[k] for k in options if len(options[k]) != 0}
        
    if len(options) == 6: 
        can_work.append(options)

can_work = can_work[0]
for n in range(0,6):
    reverse = {}
    for i in range(0,6):
        reverse[i] = {can_work[i][l][0][0] for l in can_work[i]}
    
    for i in range(0,6):
        can_work[i] = {j:can_work[i][j] for j in can_work[i] if j in reverse[(i-1)%6]}

numbers = []
for k in can_work:
    for l in can_work[k]:
        numbers.append(int(l + can_work[k][l][0][0]))
        
sum(numbers)

28684

## Problem 62

### Cubic permutations

In [48]:
import itertools

limit = 10000
cubes = [i**3 for i in range(1,limit)]
perms = dict()
permutations = 5

for c in cubes:
    tmp = list(str(c))
    tmp.sort()
    key = ''.join(tmp)
    if key not in perms:
        perms[key] = []
    perms[key].append(c)
    
for k in perms:
    if len(perms[k]) >= permutations:
        break
        
perms[k][0]

127035954683

## Problem 63

### Powerful digit counts

In [47]:
import math

count = 0
n = 1
min_pow = 1
while min_pow < 10:
    min_pow = math.ceil((10**(n-1))**(1/n))
    count += 10 - min_pow
    n += 1  

count

49

## Problem 64

### Odd period square roots

In [129]:
import math

def square_base(n):
    return int(math.sqrt(n))

def simplify(n,d):
    i = 2
    while i <= n and i <= d:
        while n % i == 0 and d % i == 0:
            n = n/i
            d = d/i
        i = i + 1
    return (int(n), int(d))

def next_fraction(n, base, nom, den_minus):
    next_nom = n - den_minus * den_minus
    nom, next_nom = simplify(nom, next_nom)
    next_part = int((base + den_minus) / next_nom)
    next_den_minus = -(den_minus - next_part * next_nom)
    return (next_nom, next_den_minus)

def period_length(n):
    previous = set()
    base = square_base(n)
    nom, den_minus = (1, base)

    while (nom, den_minus) not in previous:
        previous.add((nom, den_minus))
        nom, den_minus = next_fraction(n, base, nom, den_minus)
        
    return len(previous)

count = 0
for n in range(2, 10000):
    if math.sqrt(n) != square_base(n):
        length = period_length(n)
        if length % 2 == 1:
            count += 1
            
count

1322

## Problem 65

### Convergents of e

In [97]:
import math

parts = []
for i in range(1, 34):
    parts.extend([1,1,i*2])
parts[0] = 2
parts.append(1)

def calculate(parts):
    parts.reverse()
    num, den = (parts[0], 1)
    for p in parts[1:]:
        num, den = (den, num)
        num += den * p
    return num, den

def digit_sum(n):
    return sum([int(i) for i in list(str(n))])

fraction = calculate(parts)
digit_sum(fraction[0])

272

## Problem 66

### Diophantine equation

In [190]:
import math

def square_base(n):
    return int(math.sqrt(n))

def simplify(n,d):
    i = 2
    while i <= n and i <= d:
        while n % i == 0 and d % i == 0:
            n = n/i
            d = d/i
        i = i + 1
    return (int(n), int(d))

def next_fraction(n, base, nom, den_minus):
    next_nom = n - den_minus * den_minus
    nom, next_nom = simplify(nom, next_nom)
    next_part = int((base + den_minus) / next_nom)
    next_den_minus = -(den_minus - next_part * next_nom)
    return (next_part, next_nom, next_den_minus)

def calculate(parts):
    parts.reverse()
    num, den = (parts[0], 1)
    for p in parts[1:]:
        num, den = (den, num)
        num += den * p
    parts.reverse()
    return num, den

def find_solution(D):
    base = square_base(D)
    nom, den_minus = (1, base)
    parts = [base]
    x, y = (1, base)
    while x*x - D*y*y != 1:
        part, nom, den_minus = next_fraction(D, base, nom, den_minus)
        parts.append(part)
        x, y = calculate(parts)
    return x

solutions = {}
for D in range(2, 1000):
    sqrt = math.sqrt(D)
    if sqrt != int(sqrt):
        x = find_solution(D)
        solutions[x] = D

solutions[max(solutions)]

661

## Problem 67

### Maximum path sum II

In [257]:
with open("p067_triangle.txt") as f:
    data = f.read()
    
triangle = [[int(n) for n in row.split(' ')] for row in data.split('\n')[:-1]]

def calculate_current(level, i):
    current = triangle[level][i]
    left = triangle[level+1][i]
    right = triangle[level+1][i+1]
    return current + max(left, right)

for level in range(len(triangle)-2, -1, -1):
    for i in range(0,len(triangle[level])):
        triangle[level][i] = calculate_current(level, i)
        
triangle[0][0]

7273

## Problem 68

### Magic 5-gon ring

In [645]:
import itertools

size = 5
base_sum = sum(range(1, size * 2 + 1))
min_sum = base_sum + sum(range(1, size + 1))
max_sum = base_sum + sum(range(size + 1, size * 2 + 1))
valid_sums = [t for t in range(min_sum, max_sum+1) if t % size == 0]
double_sums = [t-base_sum for t in valid_sums]
totals = [int(t/size) for t in valid_sums]

combinations = list(itertools.combinations(range(1, min(size * 2 + 1, 10)), size))
combinations = [c for c in combinations if sum(c) in double_sums]

def is_valid(lines):
    if len({lines[l] for l in lines}) < size:
        return False
    node_counter = {}
    for l in lines:
        if l[0] not in node_counter:
            node_counter[l[0]] = 0
        if l[1] not in node_counter:
            node_counter[l[1]] = 0
        node_counter[l[0]] += 1
        node_counter[l[1]] += 1
    return len({node_counter[n] for n in node_counter}) == 1

candidates = []
for c in combinations:
    remaining = [n for n in range(1, size * 2 + 1) if n not in c]
    total = int((base_sum + sum(c))/size)
    groups = list(itertools.combinations(c, 2))
    lines = {s: total - sum(s) for s in groups if total - sum(s) in remaining}
    options = list(itertools.combinations(lines, size))
    for o in options:
        lines = {}
        for i in range(0,size):
            lines[o[i]] = total - sum(o[i])
        if not is_valid(lines):
            continue
        candidates.append(lines)


candidates = {min([c[inner] for inner in c]): c for c in candidates}
candidate = candidates[max(candidates)]

def create_rings(lines):
    rings = []
    connections = {}
    for l in lines:
        connections[l] = [m for m in lines if (l[0]==m[0] or l[0]==m[1] or l[1]==m[0] or l[1]==m[1]) and l != m]
    outers = {lines[l]:l for l in lines}
    last = outers[min(outers)]
    reverse = 1
    ring = str(lines[last]) + str(last[(0+reverse)%2]) + str(last[(1+reverse)%2])
    for i in range(0, size - 1):
        del lines[last]
        current = [l for l in connections[last] if l[0]==last[(1+reverse)%2] or l[1]==last[(1+reverse)%2]][0]
        del connections[last]
        if current[0] == last[(1+reverse)%2]:
            reverse = 0
        else:
            reverse = 1
        ring += str(lines[current]) + str(current[(0+reverse)%2]) + str(current[(1+reverse)%2])
        last = current
    return int(ring)
        
create_rings(candidate)

6531031914842725

## Problem 69

### Totient maximum

In [154]:
limit_pow = 6
limit = 10**limit_pow

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]


primes = prime_sieve(limit)

n_per_totients = [1]*(limit-1)

for p in primes:
    n_per_totients[p-2::p] = [p/(p-1)*x for x in n_per_totients[p-2::p]]

    
n_per_totients.index(max(n_per_totients))+2

510510

## Problem 70

### Totient permutation


In [202]:
limit_pow = 7
limit = 10**limit_pow

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]


primes = prime_sieve(limit)

totients = list(range(2,limit))

for p in primes:
    totients[p-2::p] = [(p-1)/p*x for x in totients[p-2::p]]
    
totients = [int(t) for t in totients]

valid = {}
n = 2
while n < limit:
    t = totients[n-2]
    if ''.join(sorted(str(n))) == ''.join(sorted(str(t))):
        valid[n] = t
    n += 1

n_per_totients = {k:k/valid[k] for k in valid}

min(n_per_totients, key=n_per_totients.get)

8319823

## Problem 71

### Ordered fractions

In [426]:
import math

target = 3/7
limit = 1000000

max_below_target = {}
for d in range(2, limit+1):
    n = math.ceil(target*d - 1)
    if n != 0:
        max_below_target[d] = n
    
def simplify(n,d):
    i = 2
    while i <= n and i <= d:
        while n % i == 0 and d % i == 0:
            n = n/i
            d = d/i
        i = i + 1
    return (int(n), int(d))
    
values = {d:max_below_target[d]/d for d in max_below_target}
max_d = max(values, key=values.get)
max_n = max_below_target[max_d]

simplify(max_n, max_d)[0]

428570

## Problem 72

### Counting fractions

In [240]:
limit_pow = 6
limit = 10**limit_pow

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]


primes = prime_sieve(limit)

totients = list(range(2,limit+1))

for p in primes:
    totients[p-2::p] = [(p-1)/p*x for x in totients[p-2::p]]
    
totients = [int(t) for t in totients]

sum(totients)

303963552391

## Problem 73

### Counting fractions in a range

In [428]:
import math

bottom = 1/3
top = 1/2
limit = 12000

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]

primes = prime_sieve(limit)

coprimes = {}
for n in range(2, limit+1):
    cop = [True]*(n-1)
    if n not in primes:
        i = 0
        while i < len(primes) and primes[i] <= n:
            p = primes[i]
            if n % p == 0:
                cop[p-1::p] = [False]*int((n-1)/p)
            i += 1
    coprimes[n] = [x for x in range(1, n) if cop[x-1]]

count = 0
for d in coprimes:
    min_n = int(bottom*d + 1)
    max_n = math.ceil(top*d - 1)
    count += len([n for n in coprimes[d] if n >= min_n and n <= max_n])
    

count

7295372

## Problem 74

### Digit factorial chains

In [459]:
limit = 1000000

digit_factorials = {'0':1}
f = 1
for n in range(1, 10):
    f *= n
    digit_factorials[str(n)] = f
    
def sum_digit_factorials(n):
    s = 0
    for i in str(n):
        s += digit_factorials[i]
    return s
    
digit_sum_factorials = {n: sum_digit_factorials(n) for n in range(1, limit)}

def digit_sum_factorial(n):
    if n not in digit_sum_factorials:
        digit_sum_factorials[n] = sum_digit_factorials(n)
    return digit_sum_factorials[n]

chain_lengths = {}

def get_chain_length(n):
    if n not in chain_lengths:
        k = n
        chain = []
        while k not in chain:
            chain.append(k)
            k = digit_sum_factorial(k)
        for i in range(0, chain.index(k)):
            chain_lengths[chain[i]] = len(chain) - i
        for i in range(chain.index(k), len(chain)):
            chain_lengths[chain[i]] = len(chain) - chain.index(k)
    return chain_lengths[n]
    

for n in range(1, limit):
    get_chain_length(n)
    

len([x for x in chain_lengths if chain_lengths[x] == 60])

402

## Problem 75

### Singular integer right triangles

In [23]:
import itertools

max_length = 1500000
limit = int(max_length**0.5)

def prime_sieve(limit):
    sieve = [True] * (int(limit/2))
    for i in range(3, int(limit**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((limit - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(limit/2)) if sieve[i]]


def generate_coprimes(limit):
    coprimes = {}
    for n in range(2, limit+1):
        cop = [True]*(n-1)
        if n not in primes:
            i = 0
            while i < len(primes) and primes[i] <= n:
                p = primes[i]
                if n % p == 0:
                    cop[p-1::p] = [False]*int((n-1)/p)
                i += 1
        coprimes[n] = [x for x in range(1, n) if cop[x-1]]
    return coprimes


def length(m,n):
    return 2*m*(m+n)


primes = prime_sieve(limit)
coprimes = generate_coprimes(limit)

coprimes = {n:[m for m in coprimes[n] if not (n % 2 == 1 and m % 2 == 1)] for n in coprimes}

lengths = {}
for m in coprimes:
    for n in coprimes[m]:
        l = length(m,n)
        i = 1
        while l*i <= max_length:
            if l*i not in lengths:
                lengths[l*i] = 0
            lengths[l*i] += 1
            i += 1

len([l for l in lengths if lengths[l] == 1])

161667

## Problem 76

### Counting summations

In [479]:
summation = 100

breakdowns_with_min = {1:{}}
for n in range(2, summation+1):
    a = 0
    breakdowns_with_min[n] = {}
    for i in range(int(n/2), 0, -1):
        a += 1
        if i in breakdowns_with_min[n-i]:
            a += breakdowns_with_min[n-i][i]
        breakdowns_with_min[n][i] = a
        
breakdowns_with_min[summation][1]

190569291

## Problem 77

### Prime summations

In [514]:
limit_pow = 6
limit = 10**limit_pow
target = 5000

def prime_sieve(limit):
    sieve = [True] * (int(limit/2))
    for i in range(3, int(limit**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((limit - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(limit/2)) if sieve[i]]

primes = prime_sieve(limit)

breakdowns_with_min = {1:{}}

n = 2
a = 0
while a < target:
    a = 0
    breakdowns_with_min[n] = {}
    for i in range(int(n/2), 0, -1):
        if i not in primes:
            continue
        if n-i in primes:
            a += 1
        if i in breakdowns_with_min[n-i]:
            a += breakdowns_with_min[n-i][i]
        breakdowns_with_min[n][i] = a
    n += 1
        
max(breakdowns_with_min)

71

## Problem 78

### Coin partitions

In [529]:
div_target = 1000000

partitions = {0:1}

def p(n):
    if n < 0:
        return 0
    if n in partitions:
        return partitions[n]
    
    p = 0
    for k in range(1, int(n**0.5+1)):     
        p1 = partition(n-k*(3*k-1)/2)
        p2 = partition(n-k*(3*k+1)/2)
                            
        if (k%2 == 1):
            p += p1 + p2
        else:
            p -= p1 + p2
    partitions[n] = p
    return p

n = 1
while p(n) % div_target != 0:
    n += 1
    
n

55374

## Problem 79

### Passcode derivation

In [86]:
with open('p079_keylog.txt') as f:
    data = f.read()
    
codes = data.split('\n')[:-1]

followedBy = {}
for c in codes:
    if c[0] not in followedBy:
        followedBy[c[0]] = set()
    followedBy[c[0]].add(c[1])
    if c[1] not in followedBy:
        followedBy[c[1]] = set()
    followedBy[c[1]].add(c[2])
    if c[2] not in followedBy:
        followedBy[c[2]] = set()
        
end = ''
lasts = [c for c in followedBy if len(followedBy[c]) == 0]
while len(lasts) != 0:
    last = lasts[0]
    end = last + end
    del followedBy[last]
    for c in followedBy:
        if last in followedBy[c]:
            followedBy[c].remove(last)
    lasts = [c for c in followedBy if len(followedBy[c]) == 0]
    
end

'73162890'

## Problem 80

### Square root digital expansion

In [87]:
acc = 0

for n in range(2,100):
    c = n
    p = 0
    if n**0.5 != int(n**0.5):
        for i in range(0, 100):
            x = 0
            while (x+1)*(20*p + (x+1)) < c:
                x += 1
            y = x*(20*p + x)
            p = p*10 + x
            c -= y
            c *= 100
            acc += x
            if c == 0:
                break

acc

40886

## Problem 81

### Path sum: two ways

In [151]:
with open('p081_matrix.txt') as f:
    data = f.read()
    
matrix = [[int(element) for element in row.split(',')] for row in data.split('\n')[:-1]]
mins = [ [0]*len(matrix) for i in range(len(matrix))]

mins[0][0] = matrix[0][0]
for i in range(len(matrix)):
    mins[0][i] = mins[0][i-1] + matrix[0][i]
    mins[i][0] = mins[i-1][0] + matrix[i][0]

for i in range(1, len(matrix)):
    for j in range(i, len(matrix)):
        mins[i][j] = matrix[i][j] + min(mins[i-1][j], mins[i][j-1])
        mins[j][i] = matrix[j][i] + min(mins[j-1][i], mins[j][i-1])
        
mins[len(matrix)-1][len(matrix)-1]

427337

## Problem 82

### Path sum: three ways

In [246]:
with open('p082_matrix.txt') as f:
    data = f.read()
    
matrix = [[int(element) for element in row.split(',')] for row in data.split('\n')[:-1]]
mins = [[0]*len(matrix) for i in range(len(matrix))]

for k in range(len(matrix)):
    mins[k][0] = matrix[k][0]
    
for i in range(1, len(matrix)):
    tmp = [matrix[j][i] + mins[j][i-1] for j in range(len(matrix))]
    sorted_tmp = sorted(tmp)
    for j in range(len(matrix)):
        mins[j][i] = tmp[j]
    
    for k in sorted_tmp:
        m = tmp.index(k)
        diffs = [0]*len(matrix)
        for j in range(m-1, -1, -1):
            diffs[j] = diffs[j+1] + matrix[j][i]
        for j in range(m+1, len(matrix)):
            diffs[j] = diffs[j-1] + matrix[j][i]
        for j in range(len(matrix)):
            if k + diffs[j] < mins[j][i]:
                mins[j][i] = k + diffs[j]        
        
min([x[len(matrix)-1] for x in mins])

260324

## Problem 83

### Path sum: four ways

In [276]:
with open('p083_matrix.txt') as f:
    data = f.read()
    
matrix = [[int(element) for element in row.split(',')] for row in data.split('\n')[:-1]]

mins = [ [0]*len(matrix) for i in range(len(matrix))]
mins[0][0] = matrix[0][0]

def neighbours_min(i,j):
    neighbours = []
    if i > 0:
        neighbours.append(mins[j][i-1])
    if j > 0:
        neighbours.append(mins[j-1][i])
    if i < len(matrix)-1:
        neighbours.append(mins[j][i+1])
    if j < len(matrix)-1:
        neighbours.append(mins[j+1][i])
    return min([x for x in neighbours if x != 0])

def set_element(i,j,k):
    mins[j][i] = k
    if i > 0:
        recalculate(i-1,j,k)
    if j > 0:
        recalculate(i,j-1,k)
    if i < len(matrix)-1:
        recalculate(i+1,j,k)
    if j < len(matrix)-1:
        recalculate(i,j+1,k)
        
def recalculate(i,j,k):
    if k+matrix[j][i] < mins[j][i]:
        set_element(i,j,k+matrix[j][i])
    
for d in range(1,len(matrix)):
    for k in range(d+1):
        set_element(k, d-k, matrix[d-k][k] + neighbours_min(k, d-k))
        
l = len(matrix)-1
for d in range(l-1, -1, -1):
    for k in range(d+1):
        set_element(l-k, l-(d-k), matrix[l-(d-k)][l-k] + neighbours_min(l-k, l-(d-k)))

mins[len(matrix)-1][len(matrix)-1]

425185